In [11]:
## ライブラリ
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [5]:
'''
東京オープデータAPIデータ取得用関数
url : https://spec.api.metro.tokyo.lg.jp/spec/search
'''
def apiJsonData(url,limit):
    # クエリパラメーター
    params = {
        'limit': limit
        }
    # POSTデータ（空のJSONオブジェクト）
    data = {}
    # ヘッダー
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
    }
    # POSTリクエストを送信してレスポンスを取得
    return requests.post(url, params=params, json=data, headers=headers)

'''
全件取得
'''
def apiJsonDataAll(url):
    # クエリパラメーター
    params = {
        'limit': 1
        }
    # POSTデータ（空のJSONオブジェクト）
    data = {}
    # ヘッダー
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
    }
    # POSTリクエストを送信してレスポンスを取得
    response = requests.post(url, params=params, json=data, headers=headers)
    if response.status_code == 200:
        json_data = response.json()
        total = json_data["total"]
        limit = json_data["limit"]
        if (total > limit) or (total < 500):
            time.sleep(1)
            response = apiJsonData(url,total)
    return response

'''
東京オープデータJSONからデータフレームに変換する関数
'''
def jsonToDataFrame(response):
    json_data = response.json()
    df = pd.DataFrame(json_data['hits'])
    return df

In [6]:
api_id = 't001001d0000000001-1db9fd22025886d3d5c9607d5c24fdf0-0'
url = f'https://service.api.metro.tokyo.lg.jp/api/{api_id}/json'
# 全件取得
response = apiJsonDataAll(url)
# レスポンスのステータスコードをチェック
if response.status_code == 200:
	print(f'OK({response})')
	df = jsonToDataFrame(response)
else:
	print('取得失敗')
df.columns

OK(<Response [200]>)


Index(['row', '展示会名', '会期(開始)', '開始曜日', '会期(終了)', '終了曜日', '利用施設', '開催時間',
       '最終日の終了時刻（●:30分前に終了　▲:1時間前に終了　■:2時間前に終了）', '来場対象者', '入場料について', '内容',
       '連絡先', 'TEL', 'URL'],
      dtype='object')

In [7]:
df[['展示会名','会期(開始)','会期(終了)','利用施設','来場対象者','URL']]

,展示会名,会期(開始),会期(終了),利用施設,来場対象者,URL
0,2024年春期セブン-イレブン商品展示会,2024/04/03,2024/04/05,南1・2,,
1,インターペット～人とペットの豊かな暮らしフェア～,2024/04/04,2024/04/07,東1-4・7・8,商談/一般,https://interpets.jp.messefrankfurt.com/tokyo/...
2,JKCサクラ・アニュアル・ショー2024,2024/04/05,2024/04/07,東5・6,一般,https://www.jkc.or.jp/
3,CYCLE MODE TOKYO 2024,2024/04/06,2024/04/07,西3・4、南3・4,商談/一般,
4,Takeoff Tokyo 2024,2024/04/10,2024/04/11,南3,商談,https://www.takeoff-tokyo.com/
...,...,...,...,...,...,...
116,電子機器トータルソリューション展2024,2024/06/12,2024/06/14,東3-6,商談,https://www.jpcashow.com/show2024/index.html
117,インテリア ライフスタイル2024,2024/06/12,2024/06/14,西1・2,商談,https://interiorlifestyle-tokyo.jp.messefrankf...
118,民間医局レジナビフェア2024東京 ～臨床・専門研修プログラム～,2024/06/16,2024/06/16,西1・2,関係者のみ,https://www.residentnavi.com/rnfair/275
119,キャリタス就活フォーラム,2024/06/22,2024/06/22,西4,一般,https://job.career-tasu.jp/


In [8]:
df.to_csv('../data/toukyou_big_2404-06.csv',index=False)

In [9]:
api_id = 't001001d0000000001-edb51fdc8dab52d1a4e190bde467b886-0'
url = f'https://service.api.metro.tokyo.lg.jp/api/{api_id}/json'
# 全件取得
response = apiJsonDataAll(url)
# レスポンスのステータスコードをチェック
if response.status_code == 200:
	print(f'OK({response})')
	df2 = jsonToDataFrame(response)
else:
	print('取得失敗')
df2.columns

OK(<Response [200]>)


Index(['row', '展示会名', '会期(開始)', '開始曜日', '会期(終了)', '終了曜日', '利用施設', '開催時間',
       '最終日の終了時刻（●:30分前に終了 ▲:1時間前に終了 ■:2時間前に終了）', '来場対象者', '入場料について', '内容',
       '連絡先', 'TEL', 'URL'],
      dtype='object')

In [10]:
df2.to_csv('../data/toukyou_big_2407-09.csv',index=False)

# K-meansクラスタリングでテキストの一覧を分類
https://qiita.com/WdknWdkn/items/eb168177ae02b2b3b5a3

In [20]:
# テキストをTF-IDFベクトル化する
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['展示会名'])

# k-meansクラスタリングを実行
num_clusters = 20  # クラスタ数
kmeans = KMeans(n_clusters=num_clusters,random_state=42)
kmeans.fit(X)

# クラスタリング結果をデータフレームに追加
df['cluster'] = kmeans.labels_

In [21]:
df[['展示会名','cluster','会期(開始)','会期(終了)','利用施設','来場対象者','URL']]

,展示会名,cluster,会期(開始),会期(終了),利用施設,来場対象者,URL
0,2024年春期セブン-イレブン商品展示会,15,2024/04/03,2024/04/05,南1・2,,
1,インターペット～人とペットの豊かな暮らしフェア～,15,2024/04/04,2024/04/07,東1-4・7・8,商談/一般,https://interpets.jp.messefrankfurt.com/tokyo/...
2,JKCサクラ・アニュアル・ショー2024,15,2024/04/05,2024/04/07,東5・6,一般,https://www.jkc.or.jp/
3,CYCLE MODE TOKYO 2024,17,2024/04/06,2024/04/07,西3・4、南3・4,商談/一般,
4,Takeoff Tokyo 2024,17,2024/04/10,2024/04/11,南3,商談,https://www.takeoff-tokyo.com/
...,...,...,...,...,...,...,...
116,電子機器トータルソリューション展2024,15,2024/06/12,2024/06/14,東3-6,商談,https://www.jpcashow.com/show2024/index.html
117,インテリア ライフスタイル2024,18,2024/06/12,2024/06/14,西1・2,商談,https://interiorlifestyle-tokyo.jp.messefrankf...
118,民間医局レジナビフェア2024東京 ～臨床・専門研修プログラム～,15,2024/06/16,2024/06/16,西1・2,関係者のみ,https://www.residentnavi.com/rnfair/275
119,キャリタス就活フォーラム,10,2024/06/22,2024/06/22,西4,一般,https://job.career-tasu.jp/
